# Controlling the Board Config

This example shows you how to edit the board's configuration over a serial connection. The steps are nearly identical over a WiFi connection. See start_webserver_over_serial.ipynb for details of starting a WiFi connection.

## Import packages

In [2]:
import sys
import os
import time

# Get the current working directory (the directory where the notebook is located)
notebook_dir = os.getcwd()

# Add the 'source' folder to the path
sys.path.append(os.path.abspath(os.path.join(notebook_dir, '..', 'source')))

# Now you can import from 'source'
from serial_controller import SerialController
from serial_command_sender import SerialCommandSender
from web_controller import WebController
from web_command_sender import WebCommandSender


## Initialize the Serial Command Sender and edit configuration

Note: make sure nothing else is holding the serial line (for instance, a micropico VSC project). You may need to disconnect and reconnect the Pico's USB serial link.

In [ ]:
# Initialize the serial controller with the appropriate port. This must be done in every cell in jupyter notebooks to avoid holding the serial line. There is no such restriction in a python script.
SERIAL_PORT = "COM6" #Find the serial port your RP Pico W is connected to, can use the MicroPico: Help > List Serial Ports in VSC's MicroPico package. Hit F1 to search for command in VSC
serial_controller = SerialController(port=SERIAL_PORT, debug=False)  # Adjust port as needed
serial_controller.connect()

# Initialize the command sender
serial_command_sender = SerialCommandSender(serial_controller)

# First, let's create a GPIO hardware so we have some config to play with
settings = {
    'pin_number': "LED", # Use the built-in LED
    'mode': 'OUT', # Set the pin as an output
    'value': 0, # Set the initial value to 1 (turn on the LED)
}
print(serial_command_sender.send_command('create', 'gpio', settings, "test_gpio")) # Create the GPIO hardware
print(serial_command_sender.send_command('get_all_config')) # Print the current configuration
print()

# Now let's turn the LED off in an untranditional way by editting the config
print(serial_command_sender.send_command('set_config', 'hardware.test_gpio.settings.value', 1)) # Set a new key-value pair
print(serial_command_sender.send_command('get_all_config')) # Print the current configuration
print()

# The LED will turn on once we apply the config
print(serial_command_sender.send_command('apply_config')) #  BUG: Apply config is not working as expected...
print(serial_command_sender.send_command('get_all_config')) # Print the current configuration
print()

time.sleep(1) # Just so you can verify the LED actually did turn on

# Now let's create a new key-value pair in the configuration
print(serial_command_sender.send_command('set_config', 'new_key', 'new_value')) # Set a new key-value pair
print(serial_command_sender.send_command('get_all_config')) # Print the current configuration
print()

# Let's edit that existing key
print(serial_command_sender.send_command('set_config', 'new_key', 12345)) # Set a new key-value pair
print(serial_command_sender.send_command('get_all_config')) # Print the current configuration
print()

# Let's save the configuration
print(serial_command_sender.send_command('save_config')) # Save the new configuration
print(serial_command_sender.send_command('get_all_config')) # Print the current configuration
print()

# Now let's delete the new key
print(serial_command_sender.send_command('delete_config_key', 'new_key')) # Delete the new key
print(serial_command_sender.send_command('get_all_config')) # Print the current configuration
print()

# We can reverse that delete by reverting back to our saved config
print(serial_command_sender.send_command('load_config')) # Load the saved configuration
print(serial_command_sender.send_command('get_all_config')) # Print the current configuration
print()

#Cleanup
print(serial_command_sender.send_command('delete', 'test_gpio'))
print(serial_command_sender.send_command('delete_config_key', 'new_key')) # Delete the new key
print(serial_command_sender.send_command('save_config')) # Save the new configuration
print(serial_command_sender.send_command('get_all_config')) # Print the current configuration
print()

serial_controller.close() #Serial controller must be closed in each cell in jupyter notebooks to avoid holding the serial line. There is no such restriction in a python script.


Connected to COM6
Created GPIO hardware with ID test_gpio.
All Configuration: {'hardware': {'test_gpio': {'settings': {'value': 0, 'hardware_id': 'test_gpio', 'pin_number': 'LED', 'start_on_init': False, 'mode': 'OUT'}, 'type': 'GPIOHardware'}}}

Set config for 'hardware.test_gpio.settings.value' to 1
All Configuration: {'hardware': {'test_gpio': {'settings': {'value': 1, 'hardware_id': 'test_gpio', 'pin_number': 'LED', 'start_on_init': False, 'mode': 'OUT'}, 'type': 'GPIOHardware'}}}

Configuration applied.
All Configuration: {'hardware': {}}

Set config for 'new_key' to new_value
All Configuration: {'new_key': 'new_value', 'hardware': {}}

Set config for 'new_key' to 12345
All Configuration: {'new_key': 12345, 'hardware': {}}

Set config for 'new_key' to 12345
All Configuration: {'new_key': 12345, 'hardware': {}}

Configuration saved.
All Configuration: {'new_key': 12345, 'hardware': {}}

Config key 'new_key' successfully removed.
All Configuration: {'hardware': {}}

Configuration lo